In [1]:
from neo4j import GraphDatabase
import openrouteservice
import pandas as pd

In [2]:
URI = "bolt://localhost:7687"  
USERNAME = "neo4j"  
PASSWORD = "Tanazi369"  
DATABASE = "neo4j" 
API_KEY = "5b3ce3597851110001cf6248b2b74b27a89f4836aa6790441684f9ea"
client = openrouteservice.Client(key=API_KEY)
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

csv_files = {
    "Vehicle": r"dataset/vehicle_updated.csv",
    "Warehouse": r"dataset/warehouse_updated.csv",
    "Customer": r"dataset/customer_updated.csv",
    "Order": r"dataset/order.csv"
}



In [3]:
def create_nodes(tx, label, properties):
        
    if "Latitude" in properties and "Longitude" in properties:
        lat, lon = properties.pop("Latitude"), properties.pop("Longitude")
        query = f"""
        CREATE (n:{label} {{ {', '.join(f'{key}: ${key}' for key in properties.keys())}, location: point({{latitude: $lat, longitude: $lon}}) }})
        """
        properties["lat"] = lat
        properties["lon"] = lon
   
    else:
        query = f"""
        CREATE (n:{label} {{ {', '.join(f'{key}: ${key}' for key in properties.keys())} }})
        """
    tx.run(query, **properties)


def load_to_neo4j():
    with driver.session(database=DATABASE) as session:
        for label, file_path in csv_files.items():
            df = pd.read_csv(file_path)            
            for _, row in df.iterrows():
                properties = {col: row[col] for col in df.columns if pd.notna(row[col])}  
                
                if label=="Order":
                    properties = {key: value for key, value in properties.items() if key not in ["Customer_ID", "Warehouse_ID"]}      

                session.execute_write(create_nodes, label, properties)

            print(f"Nodes for {label} created successfully!")


In [4]:
def relation_from_orders(tx, order_id, customer_id, warehouse_id):
    query = """
    MATCH (o:Order {Order_ID: $order_id})
    MATCH (c:Customer {Customer_ID: $customer_id})
    MATCH (w:Warehouse {Warehouse_ID: $warehouse_id})
    MERGE (o)-[:PLACED_BY]->(c)
    MERGE (o)-[:SHIPPED_FROM]->(w)
    """
    tx.run(query, order_id=order_id, customer_id=customer_id, warehouse_id=warehouse_id)


In [5]:
def connect_orders(file_path):
    df = pd.read_csv(file_path)
        
    with driver.session(database=DATABASE) as session:
        for _, row in df.iterrows():
            order_id = row["Order_ID"]
            customer_id = row["Customer_ID"]
            warehouse_id = row["Warehouse_ID"]
            session.execute_write(relation_from_orders, order_id, customer_id, warehouse_id)

        print("Orders are connected with respective warehouse and customer!")

In [6]:
def route_between_two_points(coords):
    route = client.directions(coords, profile='driving-car', format='json')
    distance = route["routes"][0]["summary"]["distance"]
    duration = route["routes"][0]["summary"]["duration"]
    return distance,duration

    

In [7]:
def route_relationship(tx, label1, id1, label2, id2, rel_type, properties):    
   
        properties_str = ", ".join(f"{key}: ${key}" for key in properties.keys()) if properties else ""     
        query = f"""
            MATCH (a:{label1} {{{label1}_ID: $id1}}), (b:{label2} {{{label2}_ID: $id2}})
            MERGE (a)-[r:{rel_type} {{ {properties_str} }}]->(b)
            """
        params = {"id1": id1, "id2": id2}    
        if properties:
            params.update(properties)
        tx.run(query, **params)


In [8]:
def generate_routes():
    
    with driver.session(database=DATABASE) as session:
        query = """
        MATCH (o:Order)-[:SHIPPED_FROM]->(w:Warehouse), (o)-[:PLACED_BY]->(c:Customer)
        RETURN o.Order_ID AS order_id, w.Warehouse_ID AS warehouse_id, w.location AS warehouse_location,
               c.Customer_ID AS customer_id, c.location AS customer_location
        """
        
        orders = session.execute_read(lambda tx: 
            [{"order_id": record["order_id"], 
              "warehouse_id": record["warehouse_id"], 
              "warehouse_location": record["warehouse_location"], 
              "customer_id": record["customer_id"], 
              "customer_location": record["customer_location"]}
             for record in tx.run(query)]
        )

        failed_connections=[]
        
        for order in orders:
            warehouse_location = order["warehouse_location"]
            customer_location = order["customer_location"]
            warehouse_id=order["warehouse_id"]
            customer_id=order["customer_id"]
            
            warehouse_coords = (warehouse_location.x, warehouse_location.y)  
            customer_coords = (customer_location.x, customer_location.y)  
            coords = [warehouse_coords, customer_coords]           
                
            try:
                distance,duration=route_between_two_points(coords)
                distance_km = round(distance / 1000, 4)  
                duration_hr = round(duration / 3600, 4)
                session.execute_write(route_relationship, "Warehouse",warehouse_id,"Customer", customer_id,"DELIVERS_TO",{"distance": distance_km, "duration": duration_hr})
                print(f"Connected Warehouse {warehouse_id} -> Customer {customer_id} [Distance: {distance_km}km, Duration: {duration_hr}hr]")
            
            except Exception as e:
                print(f"Failed to connect {warehouse_id} -> {customer_id}: {str(e)}")
                failed_connections.append({"warehouse_id": warehouse_id, "customer_id": customer_id})
            
        
        print()        
        print(failed_connections)



In [9]:
def assign_vehicles_to_nearest_warehouse():
    with driver.session(database=DATABASE) as session:
        
        vehicles = session.execute_read(lambda tx: [
            {"vehicle_id": record["vehicle_id"], "location": record["location"]}
            for record in tx.run("MATCH (v:Vehicle) RETURN v.Vehicle_ID AS vehicle_id, v.location AS location")
        ])

        warehouses = session.execute_read(lambda tx: [
            {"warehouse_id": record["warehouse_id"], "location": record["location"]}
            for record in tx.run("MATCH (w:Warehouse) RETURN w.Warehouse_ID AS warehouse_id, w.location AS location")
        ])

        for vehicle in vehicles:
            v_loc = vehicle["location"]
            v_coords = (v_loc.x, v_loc.y)

            min_distance = float("inf")
            closest_warehouse = None
            duration_for_closest = None

            for warehouse in warehouses:
                w_loc = warehouse["location"]
                w_coords = (w_loc.x, w_loc.y)

                try:
                    distance, duration = route_between_two_points([v_coords, w_coords])
                    if distance < min_distance:
                        min_distance = distance
                        duration_for_closest = duration
                        closest_warehouse = warehouse
                except:
                    continue

            if closest_warehouse:
                warehouse_id = closest_warehouse["warehouse_id"]
                vehicle_id = vehicle["vehicle_id"]
                distance_km = round(min_distance / 1000, 2)
                duration_hr = round(duration_for_closest / 3600, 2)  

                def create_route(tx):
                    query = """
                    MATCH (v:Vehicle {Vehicle_ID: $vehicle_id})
                    MATCH (w:Warehouse {Warehouse_ID: $warehouse_id})
                    MERGE (v)-[r:ROUTE_TO]->(w)
                    SET r.vehicle_id = $vehicle_id,
                        r.order = 1,
                        r.distance = $distance_km,
                        r.duration = $duration_hr,
                        r.time_taken = $duration_hr
                    """
                    tx.run(query, vehicle_id=vehicle_id, warehouse_id=warehouse_id, distance_km=distance_km, duration_hr=duration_hr)

                session.execute_write(create_route)
                print(f"Vehicle {vehicle_id} routed to Warehouse {warehouse_id} [Distance: {distance_km} km, Duration: {duration_hr} hr]")




In [10]:
def create_projection(distance_threshold_km=30):
    with driver.session(database=DATABASE) as session:
        query = """
        CALL gds.graph.project.cypher(
            'localDeliveryGraph',
            $nodeQuery,
            $relQuery,
            { parameters: { threshold: $threshold } }
        )
        YIELD graphName, nodeCount, relationshipCount, projectMillis
        RETURN graphName, nodeCount, relationshipCount, projectMillis
        """
        
        node_query = """
        MATCH (n) 
        WHERE n:Warehouse OR n:Customer 
        RETURN id(n) AS id, labels(n) AS labels
        """
        
        rel_query = """
        MATCH (w:Warehouse)-[r:DELIVERS_TO]->(c:Customer) 
        WHERE r.distance <= $threshold 
        RETURN id(w) AS source, id(c) AS target, r.distance AS distance
        """
        
        result = session.run(query, 
                             nodeQuery=node_query,
                             relQuery=rel_query,
                             threshold=distance_threshold_km)
        stats = result.single()
        
        print(f"Subgraph 'localDeliveryGraph' created (distance ≤ {distance_threshold_km}km)")
        print(f"Nodes: {stats['nodeCount']}, Relationships: {stats['relationshipCount']}")
        return stats


In [11]:
def apply_wcc_clustering():
    with driver.session(database=DATABASE) as session:
       
        result = session.run("""
            CALL gds.wcc.write('localDeliveryGraph', {
                writeProperty: 'local_cluster'
            })
            YIELD nodePropertiesWritten, componentCount
        """)
        summary = result.single()
        print(f"Clustering done. Wrote {summary['nodePropertiesWritten']} node properties into 'local_cluster'.")
        print(f"Found {summary['componentCount']} clusters.")

       
        session.run("CALL gds.graph.drop('localDeliveryGraph')")
        print("In-memory graph 'localDeliveryGraph' dropped.")


In [12]:
def get_cluster_node_count():
    query = """
    MATCH (n)-[r:DELIVERS_TO]->(m)
    WHERE n.local_cluster = m.local_cluster
    RETURN n.local_cluster AS clusterId, collect(n) AS nodes
    """
    
    
    with driver.session() as session:
        result = session.run(query)
        
       
        for record in result:
            cluster_id = record["clusterId"]
            nodes = record["nodes"]
            
            
            print(f"Cluster ID: {cluster_id}")
            for node in nodes:
                print(f"  Node: {node}")
            print("-" * 50)

In [13]:
def local_cluster():
    
    
    with driver.session(database=DATABASE) as session:
        
        query = """
    MATCH (c:Customer)
    RETURN c.Customer_ID AS customer_id, 
           c.location AS location, 
           c.local_cluster AS cluster_id
        """
        customers = session.execute_read(lambda tx: 
        [ 
            {
            "customer_id": record["customer_id"],
            "location": record["location"],
            "cluster_id": record["cluster_id"]
            }
            for record in tx.run(query)
        ]
        )

        from collections import defaultdict

        
        clusters = defaultdict(list)
        for customer in customers:
            clusters[customer["cluster_id"]].append(customer)

        failed_connections = []

        for cluster_id, cluster_customers in clusters.items():
            for i in range(len(cluster_customers)):
                for j in range(i + 1, len(cluster_customers)):
                    cust1 = cluster_customers[i]
                    cust2 = cluster_customers[j]

                    coords1 = (cust1["location"].x, cust1["location"].y)
                    coords2 = (cust2["location"].x, cust2["location"].y)
                    coords = [coords1, coords2]

                    try:
                        distance, duration = route_between_two_points(coords)
                        distance_km = round(distance / 1000, 4)
                        duration_hr = round(duration / 3600, 4)

                        
                        session.execute_write(
                            route_relationship, "Customer", cust1["customer_id"], "Customer", cust2["customer_id"], 
                            "IN_CLUSTER", {"distance": distance_km, "duration": duration_hr}
                        )
                        session.execute_write(
                            route_relationship, "Customer", cust2["customer_id"], "Customer", cust1["customer_id"], 
                            "IN_CLUSTER", {"distance": distance_km, "duration": duration_hr}
                        )

                        print(f"Connected C{cust1['customer_id']} <-> C{cust2['customer_id']} [Cluster {cluster_id} | Distance: {distance_km}km | Duration: {duration_hr}hr]")

                    except Exception as e:
                        print(f"Failed to connect C{cust1['customer_id']} <-> C{cust2['customer_id']}: {str(e)}")
                        failed_connections.append({
                            "customer1": cust1["customer_id"],
                            "customer2": cust2["customer_id"]
                        })
                        
        print()
        print(failed_connections)


In [14]:
def local_routes_with_tsp():
    
    def get_edge_data(session, from_id, to_id, is_from_warehouse=False,to_warehouse=False):
                
        if is_from_warehouse:
            query = """
                MATCH (:Warehouse {Warehouse_ID: $from})-[r:DELIVERS_TO]->(:Customer {Customer_ID: $to})
                RETURN r.distance AS dist, r.duration AS dur
            """
            
        elif to_warehouse:
            query = """
                MATCH (:Warehouse {Warehouse_ID: $to})-[r:DELIVERS_TO]->(:Customer {Customer_ID: $from})
                RETURN r.distance AS dist, r.duration AS dur
            """
        else:
            query = """
                MATCH (:Customer {Customer_ID: $from})-[r:IN_CLUSTER]->(:Customer {Customer_ID: $to})
                RETURN r.distance AS dist, r.duration AS dur
            """
        result = session.run(query, {"from": from_id, "to": to_id})
        record = result.single()
        return (record["dist"], record["dur"]) if record else (float("inf"), float("inf"))

    with driver.session(database=DATABASE) as session:
        cluster_ids_result = session.run("""
            MATCH (n) WHERE n.local_cluster IS NOT NULL
            RETURN DISTINCT n.local_cluster AS cluster_id
        """)
        cluster_ids = [record["cluster_id"] for record in cluster_ids_result]

        for cluster_id in cluster_ids:
            result = session.run("""
                MATCH (w:Warehouse)<-[:SHIPPED_FROM]-(o:Order)-[:PLACED_BY]->(c:Customer)
                WHERE w.local_cluster = $cid AND c.local_cluster = $cid
                RETURN DISTINCT w.Warehouse_ID AS warehouse_id, collect(DISTINCT c.Customer_ID) AS customers
            """, cid=cluster_id)

            record = result.single()
            if not record:
                continue

            warehouse_id = record["warehouse_id"]
            customers = record["customers"]

            vehicle_result = session.run("""
                MATCH (v:Vehicle)-[:ROUTE_TO]->(w:Warehouse {Warehouse_ID: $wid})
                RETURN v.Vehicle_ID AS vehicle_id
            """, wid=warehouse_id)

            vehicle_record = vehicle_result.single()
            if not vehicle_record:
                continue

            vehicle_id = vehicle_record["vehicle_id"]

            order_result = session.run("""
                MATCH (:Vehicle {Vehicle_ID: $vid})-[r:ROUTE_TO]->()
                RETURN coalesce(max(r.order), 0) AS max_order,
                       coalesce(max(r.time_taken), 0) AS max_time
            """, vid=vehicle_id)

            vehicle_data = order_result.single()
            order_counter = vehicle_data["max_order"] + 1
            cumulative_duration = vehicle_data["max_time"] if vehicle_data else 0

          
            unvisited = set(customers)
            route = []
            current = warehouse_id
            is_warehouse = True

            while unvisited:
                best_next = None
                min_dist = float('inf')
                for candidate in unvisited:
                    dist, _ = get_edge_data(session, current, candidate, is_warehouse)
                    if dist < min_dist:
                        min_dist = dist
                        best_next = candidate
                
                if best_next is None:
                    break
                route.append(best_next)
                unvisited.remove(best_next)
                current = best_next
                is_warehouse = False

            full_route = [("Warehouse", warehouse_id)] + [("Customer", cid) for cid in route] + [("Warehouse", warehouse_id)]
            full_route.append(("Warehouse", warehouse_id))
            
            for i in range(len(full_route) - 1):
                label1, id1 = full_route[i]
                label2, id2 = full_route[i + 1]
                is_from_warehouse = label1 == "Warehouse"
                to_warehouse = label2 == "Warehouse"
                dist, dur = get_edge_data(session, id1, id2, is_from_warehouse,to_warehouse)
                if dist == float('inf') or dur == float('inf'):
                        continue

                duration_hours = dur
                cumulative_duration += duration_hours

                session.execute_write(
                    route_relationship,
                    label1, id1,
                    label2, id2,
                    "ROUTE_TO",
                    {
                        "distance": round(dist, 4),
                        "duration": round(duration_hours, 4),
                        "order": order_counter,
                        "vehicle_id": vehicle_id,
                        "time_taken": round(cumulative_duration, 4)
                    }
                )

                if label2 == "Customer":
                    session.run("""
                        MATCH (o:Order)-[:PLACED_BY]->(c:Customer {Customer_ID: $cust_id})
                        SET c.time_taken = $time_taken,
                            o.Status = 'scheduled',
                            o.time_taken = $time_taken
                    """, {
                        "cust_id": id2,
                        "time_taken": round(cumulative_duration, 4)
                    })

                print(f"{label1} {id1} -> {label2} {id2} [Vehicle: {vehicle_id}]")
                order_counter += 1
        
        
        session.run("""
            MATCH ()-[r:IN_CLUSTER]->()
            DELETE r
        """)        
            
 

    
            


In [15]:
load_to_neo4j()

Nodes for Vehicle created successfully!
Nodes for Warehouse created successfully!
Nodes for Customer created successfully!
Nodes for Order created successfully!


In [16]:
connect_orders(csv_files["Order"])

Orders are connected with respective warehouse and customer!


In [17]:
generate_routes()

Connected Warehouse W001 -> Customer C006 [Distance: 331.8729km, Duration: 4.4134hr]
Connected Warehouse W001 -> Customer C007 [Distance: 479.2303km, Duration: 6.2792hr]
Connected Warehouse W001 -> Customer C012 [Distance: 467.0068km, Duration: 6.0661hr]
Connected Warehouse W001 -> Customer C013 [Distance: 647.4129km, Duration: 8.2888hr]
Connected Warehouse W001 -> Customer C023 [Distance: 10.371km, Duration: 0.3079hr]
Connected Warehouse W001 -> Customer C024 [Distance: 649.9511km, Duration: 8.407hr]
Connected Warehouse W001 -> Customer C025 [Distance: 325.2915km, Duration: 4.2855hr]
Connected Warehouse W002 -> Customer C001 [Distance: 16.9968km, Duration: 0.4145hr]
Connected Warehouse W002 -> Customer C004 [Distance: 575.6503km, Duration: 7.5724hr]
Connected Warehouse W002 -> Customer C008 [Distance: 570.5281km, Duration: 7.5204hr]
Connected Warehouse W002 -> Customer C018 [Distance: 327.6005km, Duration: 4.3245hr]
Connected Warehouse W002 -> Customer C019 [Distance: 40.7345km, Durat

c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for 

Connected Warehouse W005 -> Customer C015 [Distance: 194.2106km, Duration: 3.1022hr]
Connected Warehouse W005 -> Customer C017 [Distance: 333.8207km, Duration: 4.224hr]
Connected Warehouse W005 -> Customer C029 [Distance: 3.2795km, Duration: 0.0633hr]
Connected Warehouse W005 -> Customer C030 [Distance: 4.7868km, Duration: 0.074hr]

[]


In [18]:
assign_vehicles_to_nearest_warehouse()

Vehicle V001 routed to Warehouse W005 [Distance: 3.53 km, Duration: 0.08 hr]
Vehicle V002 routed to Warehouse W001 [Distance: 8.27 km, Duration: 0.22 hr]
Vehicle V003 routed to Warehouse W002 [Distance: 3.23 km, Duration: 0.1 hr]
Vehicle V004 routed to Warehouse W003 [Distance: 4.54 km, Duration: 0.09 hr]
Vehicle V005 routed to Warehouse W004 [Distance: 14.02 km, Duration: 0.35 hr]


In [19]:
create_projection(distance_threshold_km=30)  
 

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('gds.graph.project.cypher' has been replaced by 'gds.graph.project Cypher projection as an aggregation function')} {position: line: 2, column: 9, offset: 9} for query: "\n        CALL gds.graph.project.cypher(\n            'localDeliveryGraph',\n            $nodeQuery,\n            $relQuery,\n            { parameters: { threshold: $threshold } }\n        )\n        YIELD graphName, nodeCount, relationshipCount, projectMillis\n        RETURN graphName, nodeCount, relationshipCount, projectMillis\n        "


Subgraph 'localDeliveryGraph' created (distance ≤ 30km)
Nodes: 35, Relationships: 7


<Record graphName='localDeliveryGraph' nodeCount=35 relationshipCount=7 projectMillis=267>

In [20]:
apply_wcc_clustering()

Clustering done. Wrote 35 node properties into 'local_cluster'.
Found 28 clusters.
In-memory graph 'localDeliveryGraph' dropped.


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL gds.graph.drop('localDeliveryGraph')"


In [21]:
get_cluster_node_count()

Cluster ID: 0
  Node: <Node element_id='4:61291bbc-605c-46a6-85b6-16fe130a930e:5' labels=frozenset({'Warehouse'}) properties={'Address': 'No. 23, Anna Salai, Chennai, Tamil Nadu, 600002', 'Warehouse_ID': 'W001', 'location': POINT(80.2459951 13.0350576), 'local_cluster': 0, 'Name': 'Chennai Central Hub'}>
--------------------------------------------------
Cluster ID: 1
  Node: <Node element_id='4:61291bbc-605c-46a6-85b6-16fe130a930e:6' labels=frozenset({'Warehouse'}) properties={'Address': '45/1, MG Road, Bengaluru, Karnataka, 560001', 'Warehouse_ID': 'W002', 'location': POINT(77.6046565 12.9759907), 'local_cluster': 1, 'Name': 'Bengaluru Distribution'}>
--------------------------------------------------
Cluster ID: 2
  Node: <Node element_id='4:61291bbc-605c-46a6-85b6-16fe130a930e:7' labels=frozenset({'Warehouse'}) properties={'Address': 'Sayyaji Rao Road, Agrahara, Chamrajpura, Mysuru, Karnataka 570001', 'Warehouse_ID': 'W003', 'location': POINT(76.6393 12.2979), 'local_cluster': 2, '

In [22]:
local_cluster()

Connected CC003 <-> CC021 [Cluster 2 | Distance: 24.9651km | Duration: 0.4575hr]
Connected CC009 <-> CC029 [Cluster 4 | Distance: 5.4674km | Duration: 0.1223hr]
Connected CC009 <-> CC030 [Cluster 4 | Distance: 8.4521km | Duration: 0.1542hr]
Connected CC029 <-> CC030 [Cluster 4 | Distance: 3.0628km | Duration: 0.0663hr]

[]


In [23]:
local_routes_with_tsp()

Warehouse W001 -> Customer C023 [Vehicle: V002]
Customer C023 -> Warehouse W001 [Vehicle: V002]
Warehouse W002 -> Customer C001 [Vehicle: V003]
Customer C001 -> Warehouse W002 [Vehicle: V003]
Warehouse W003 -> Customer C021 [Vehicle: V004]
Customer C021 -> Customer C003 [Vehicle: V004]
Customer C003 -> Warehouse W003 [Vehicle: V004]
Warehouse W005 -> Customer C029 [Vehicle: V001]
Customer C029 -> Customer C030 [Vehicle: V001]
Customer C030 -> Customer C009 [Vehicle: V001]
Customer C009 -> Warehouse W005 [Vehicle: V001]
